In [1]:
import numpy as np
import pandas as pd
label = np.random.choice([1,2],replace=True,size=(1000,1))
nums = np.random.normal(size=(1000,1))
cats = np.random.choice([1,2,3,4,6],replace=True,size=(1000,1))

In [2]:
x = np.concatenate([nums,cats,label],axis=1)
train = pd.DataFrame(x,columns=["num_1","cat_1","target"])
train["cat_1"] = train["cat_1"].astype("object")

In [3]:

from sklearn.preprocessing import LabelEncoder
nunique = train.nunique()
types = train.dtypes

categorical_columns = []
categorical_dims =  {}
for col in train.columns:
    if types[col] == 'object' or nunique[col] < 200:
        print(col, train[col].nunique())
        l_enc = LabelEncoder()
        train[col] = train[col].fillna("VV_likely")
        train[col] = l_enc.fit_transform(train[col].values)
        categorical_columns.append(col)
        categorical_dims[col] = len(l_enc.classes_)
    else:
        train.fillna(train[col].mean(), inplace=True)

cat_1 5
target 2


In [4]:
unused_feat = ['temp']
target = "target"

features = [ col for col in train.columns if col not in unused_feat+[target]] 

cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns]

cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]

In [6]:
from pytorch_tabnet.tab_model import TabNetClassifier
import torch
clf = TabNetClassifier(cat_idxs=cat_idxs,
                       cat_dims=cat_dims,
                       cat_emb_dim=1,
                       optimizer_fn=torch.optim.Adam,
                       optimizer_params=dict(lr=2e-2),
                       scheduler_params={"step_size":50, # how to use learning rate scheduler
                                         "gamma":0.9},
                       scheduler_fn=torch.optim.lr_scheduler.StepLR,
                       mask_type='entmax' # "sparsemax"
                      )

Device used : cpu


In [8]:
X_train = train[features].values
y_train = train[target].values

max_epochs = 1000 if not os.getenv("CI", False) else 2

In [13]:
clf.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train)],
    eval_name=['train'],
    eval_metric=['auc'],
    max_epochs=max_epochs, patience=20,
    batch_size=1024, virtual_batch_size=128,
    num_workers=0,
    weights=1,
    drop_last=False
)

epoch 0  | loss: 0.76517 | train_auc: 0.55284 |  0:00:00s
epoch 1  | loss: 0.7731  | train_auc: 0.54458 |  0:00:00s
epoch 2  | loss: 0.70236 | train_auc: 0.53682 |  0:00:00s
epoch 3  | loss: 0.70894 | train_auc: 0.54361 |  0:00:00s
epoch 4  | loss: 0.70166 | train_auc: 0.5192  |  0:00:00s
epoch 5  | loss: 0.69576 | train_auc: 0.51546 |  0:00:00s
epoch 6  | loss: 0.6938  | train_auc: 0.51186 |  0:00:00s
epoch 7  | loss: 0.69338 | train_auc: 0.52758 |  0:00:00s
epoch 8  | loss: 0.694   | train_auc: 0.53428 |  0:00:01s
epoch 9  | loss: 0.68927 | train_auc: 0.53256 |  0:00:01s
epoch 10 | loss: 0.68658 | train_auc: 0.55032 |  0:00:01s
epoch 11 | loss: 0.68934 | train_auc: 0.54836 |  0:00:01s
epoch 12 | loss: 0.69459 | train_auc: 0.55135 |  0:00:01s
epoch 13 | loss: 0.685   | train_auc: 0.55982 |  0:00:01s
epoch 14 | loss: 0.68631 | train_auc: 0.56228 |  0:00:01s
epoch 15 | loss: 0.68595 | train_auc: 0.55759 |  0:00:01s
epoch 16 | loss: 0.6888  | train_auc: 0.55071 |  0:00:01s
epoch 17 | los

In [14]:
clf.network

TabNet(
  (embedder): EmbeddingGenerator(
    (embeddings): ModuleList(
      (0): Embedding(5, 1)
    )
  )
  (tabnet): TabNetNoEmbeddings(
    (initial_bn): BatchNorm1d(2, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
    (encoder): TabNetEncoder(
      (initial_bn): BatchNorm1d(2, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
      (initial_splitter): FeatTransformer(
        (shared): GLU_Block(
          (shared_layers): ModuleList(
            (0): Linear(in_features=2, out_features=32, bias=False)
            (1): Linear(in_features=16, out_features=32, bias=False)
          )
          (glu_layers): ModuleList(
            (0): GLU_Layer(
              (fc): Linear(in_features=2, out_features=32, bias=False)
              (bn): GBN(
                (bn): BatchNorm1d(32, eps=1e-05, momentum=0.02, affine=True, track_running_stats=True)
              )
            )
            (1): GLU_Layer(
              (fc): Linear(in_features=16, out_fea